# general_demographics-pca.ipynb

### CSc-59867 - Senior Design - Prof. Etemadpour

* Purpose: exploratory data analysis on the NYC census demographics dataset from NYC Open Data
* Date started: 2021-02-26
* Authors: Xin Chen, Ian S. McBride, Lifu Tao

In [ ]:
import matplotlib.patheffects as PathEffects
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

import time

### Access dataset

In [ ]:
# Read datasets into a DataFrame
# * Data came from:
#     * https://data.cityofnewyork.us/City-Government/Census-Demographics-at-the-NYC-Community-District-/5unr-w4sc
# * Data was reorganized because the original spreadsheet wasn't formatted well
# * Discussion of variable meanings
#     * https://www.2020census.gov/en/conducting-the-count/gq.html
df = pd.read_csv('data/general_demographics/nyc_demographics_2010_by_cd.csv', index_col='variable_edit')

### Display data

In [ ]:
df.drop(columns=['variable_orig'], inplace=True)
df = df.transpose()
df = df.astype(float)
df.sort_index(inplace=True)
display('Transposed, sorted dataframe')
display(df.info())
display(df.head())

### Create helpers

In [ ]:
columns = df.columns.to_list()
columns_nested = {
    'total': ['total-population'],
    'race': [
         'race-white-nonhispanic',
         'race-black-nonhispanic',
         'race-asian-and-pacific-islander-nonhispanic',
         'race-other-nonhispanic',
         'race-two-or-more-races-nonhispanic',
         'race-hispanic-origin',
    ],
    'sex': [
         'sex-female',
         'sex-male',
    ],
    'age': [
         'age-under-5-years',
         'age-5-to-9-years',
         'age-10-to-14-years',
         'age-15-to-19-years',
         'age-20-to-24-years',
         'age-25-to-44-years',
         'age-45-to-64-years',
         'age-65-years-and-over',
         'age-nonminors',        
    ],
    'persons-living-in': [
         'persons-living-in-households',
         'persons-living-in-group-quarters',
    ],
    'persons-living-in-family-households': [
         'persons-living-in-family-households',
         'persons-living-in-family-households-householder',
         'persons-living-in-family-households-spouse',
         'persons-living-in-family-households-own-child-under-18-years',
         'persons-living-in-family-households-other-relatives',
         'persons-living-in-family-households-nonrelatives',
    ],
    'persons-living-in-nonfamily-households': [
         'persons-living-in-nonfamily-household',
         'persons-living-in-nonfamily-household-householder',
         'persons-living-in-nonfamily-household-senior-alone',
         'persons-living-in-nonfamily-household-nonrelatives',  
    ],
    'households': [
         'households',
         'households-family-households',
         'households-nonfamily-households',
         'households-with-seniors',
    ],
    'family-households': [
         'family-households-married-couple-family',
         'family-households-married-couple-with-minor-child',
         'family-households-female-householder-no-husband-present',
         'family-households-female-householder-with-minor-child',
         'family-households-male-householder-no-wife-present',
         'family-households-male-householder-with-minor-child',
    ],
    'persons-per': [
         'persons-per-family',
         'persons-per-household',        
    ],
    'housing-units': [
         'housing-units',
         'housing-units-occupied',
         'housing-units-occupied-renter',
         'housing-units-occupied-owner',
         'housing-unit-size-1-person',
         'housing-unit-size-2-person',
         'housing-unit-size-3-person',
         'housing-unit-size-4-person',
         'housing-unit-size-5-persons-and-over',
         'housing-units-by-age-of-householder-15-to-24-years',
         'housing-units-by-age-of-householder-25-to-44-years',
         'housing-units-by-age-of-householder-45-to-64-years',
         'housing-units-by-age-of-householder-65-years-and-over',
    ],
}

In [ ]:
# Helper for displaying CD numbers
cd_boro_dict = {
    1: 'Manhattan',
    2: 'Bronx',
    3: 'Brooklyn',
    4: 'Queens',
    5: 'Staten Island',
}
def cd_to_name(cd):
    cd_str = str(cd)
    if not len(cd_str) == 3:
        raise Exception('incorrect length of CD')
    cd_num = cd_str[1:]
    boro = cd_boro_dict.get(int(cd_str[0]))
    if not boro:
        raise Exception('incorrect borough prefix in CD')
    return f'{boro} CD-{cd_num}'
def cd_to_borough_num(cd):
    cd_str = str(cd)
    if not len(cd_str) == 3:
        raise Exception('incorrect length of CD')
    cd_num = cd_str[1:]
    boro = cd_boro_dict.get(int(cd_str[0]))
    if not boro:
        raise Exception('incorrect borough prefix in CD')
    return int(cd_str[0]) - 1
def cd_to_borough(cd):
    cd_str = str(cd)
    if not len(cd_str) == 3:
        raise Exception('incorrect length of CD')
    cd_num = cd_str[1:]
    boro = cd_boro_dict.get(int(cd_str[0]))
    if not boro:
        raise Exception('incorrect borough prefix in CD')
    return boro

### Plot correlation

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1, 1, 1)

corr_matrix = df.corr().values
_ = sns.heatmap(corr_matrix, ax=ax)
_ = ax.set_title('Correlation matrix')

### Select features, scale data, run PCA

In [ ]:
# Helper functions
def select_feat(df, features_nested, exclude_group=None):
    result = df.copy()
    if not exclude_group:
        pass
    elif type(exclude_group) == str:
        result.drop(columns=features_nested[exclude_group], inplace=True)
    elif type(exclude_group) == list:
        [result.drop(columns=features_nested[group], inplace=True) for group in exclude_group]
    else:
        raise Exception('bad type of exclude_group parameter')
    
    return result

def scale_data(df):
    scaler = StandardScaler()
    X = scaler.fit_transform(df.values)
    return X

def run_pca(X, feature_names, n_components=5):
    # Run and time the PCA
    time_start = time.time()
    pca = PCA(n_components=n_components)
    pca_result = pca.fit_transform(X)
    elapsed_time = time.time() - time_start
    print(f'PCA done! Time elapsed: {elapsed_time:0.2} seconds')
    explained_var = pca.explained_variance_ratio_
    print('Cumulative explained variance ratio', np.cumsum(explained_var))
    print('Shape of components:', pca.components_.shape)

    # Plot explained variance ratio
    indicies = range(1, len(explained_var) + 1)
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(1, 1, 1)
    _ = plt.plot(
        indicies,
        explained_var,
        alpha=0.5,
        marker='o',
    )
    _ = ax.set_xlabel('PC')
    _ = ax.set_ylabel('Explained variance ratio')
    _ = ax.set_xticks(indicies)
    _ = ax.set_title('PCA Results')
    
    # Plot principle components as mixtures of features in matrix heatmap
    # Inspired by the tutorial:
    # https://towardsdatascience.com/dive-into-pca-principal-component-analysis-with-python-43ded13ead21
    ticks = list(range(n_components))
    labels = [f'PC-{i}' for i in range(1, n_components + 1)]
    _ = plt.matshow(pca.components_.T, cmap='coolwarm')
    _ = plt.yticks(range(len(feature_names)), feature_names, ha='right')
    _ = plt.xticks(ticks, labels, fontsize=10, rotation=65)
    _ = plt.colorbar()
    _ = plt.title('Mixture of features in each PC')
    
    # Plot pca results as scatter matrix
    component_names = [f'PC-{i + 1}' for i in ticks]
    pca_df = pd.DataFrame(columns=component_names)
    for i, name in zip(ticks, component_names):
        pca_df[name] = pca_result[:, i]

    _ = pd.plotting.scatter_matrix(pca_df, figsize=(12, 12))
    _ = plt.suptitle(f'Scatter matrix with {n_components} principle components')

In [ ]:
# Use PCA helpers
selected_df = select_feat(df, columns_nested)
X = scale_data(selected_df)
run_pca(X, selected_df.columns, n_components=5)

### Run t-SNE

In [ ]:
def run_tsne(x, classes, n_components=2):
    # Execute t-SNE
    RS = 123
    time_start = time.time()
    tsne_results = TSNE(random_state=RS, n_components=n_components).fit_transform(x)
    elapsed_time = time.time() - time_start
    print(f't-SNE done! Time elapsed: {elapsed_time:0.2} seconds')

    # Put t-SNE results into a dataframe
    df_tsne = pd.DataFrame(
        data={
            'x': tsne_results[:, 0],
            'y': tsne_results[:, 1],
            'class': classes
        }
    )

    # Create axis
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')

    # Plot each class
    for cla in np.unique(classes):
        ax.scatter(
            x = df_tsne[df_tsne['class'] == cla]['x'],
            y = df_tsne[df_tsne['class'] == cla]['y'],
            label = cla
        )

    ax.legend()
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

In [ ]:
# Use t-SNE helper with boroughs as classes
selected_df = select_feat(df, columns_nested)
X = scale_data(selected_df)
classes = np.array([cd_to_borough(cd) for cd in df.index])
run_tsne(X, classes)